<a href="https://colab.research.google.com/github/erksubasi/transformers/blob/main/FunctimeTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install functime

In [4]:
import polars as pl
from functime.cross_validation import train_test_split
from functime.forecasting import lightgbm
from functime.metrics import mase

# Load example data
y = pl.read_parquet("https://github.com/descendant-ai/functime/raw/main/data/commodities.parquet")
entity_col, time_col = y.columns[:2]

# Time series split
y_train, y_test = y.pipe(train_test_split(test_size=3))

# Fit-predict
model = lightgbm(freq="1mo", lags=24, max_horizons=3, strategy="ensemble")
model.fit(y=y_train)
y_pred = model.predict(fh=3)

# functime ❤️ functional design
# fit-predict in a single line
y_pred = lightgbm(freq="1mo", lags=24)(y=y_train, fh=3)

# Score forecasts in parallel
scores = mase(y_true=y_test, y_pred=y_pred, y_train=y_train)

/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
Fitting direct forecasters:: 100%|██████████| 3/3 [00:02<00:00,  1.45it/s]


In [5]:
scores

commodity_type,mase
str,f64
"""Rice, Thai 25%…",3.032552
"""Sugar, EU""",174.755501
"""Groundnuts""",3.603058
"""Coffee, Arabic…",7.963714
"""Crude oil, WTI…",8.224842
"""Sugar, US""",76.090066
"""Coconut oil""",8.248196
"""Rice, Thai 5% …",3.581516
"""Cocoa""",0.790271
